In [2]:
from pyiron_workflow_atomistics.bulk import eos_volume_scan, get_equil_lat_param, get_bulk_structure

from tensorpotential.calculator import TPCalculator

from ase.build import bulk

import numpy as np

from pyiron_workflow import Workflow
from pyiron_workflow_atomistics.calculator import calculate_structure_node
from pyiron_workflow_atomistics.gb.gb_code.constructor import construct_GB_from_GBCode

import pyiron_workflow as pwf
%load_ext autoreload
%autoreload 2

In [ ]:
#LAMMPS executable.

import os
import subprocess

from ase.build import bulk
from pyiron_lammps import parse_lammps_output_files, write_lammps_structure

lammps_input_script = """\
units metal
dimension 3
boundary p p p
atom_style atomic

read_data lammps_with_masses.data

pair_style grace
pair_coeff * * /cmmc/u/hmai/2025_04_29_FeGB_Segregation_Workflows/4_GPU_run1/seed/1/saved_model/ Fe

# per‐atom potential energy
compute eng all pe/atom

# dump every 10 steps: coords, forces, velocities, AND per‐atom energy
dump 1 all custom 10 dump.out id type xsu ysu zsu fx fy fz vx vy vz c_eng
dump_modify 1 sort id format line "%d %d %20.15g %20.15g %20.15g %20.15g %20.15g %20.15g %20.15g %20.15g %20.15g %20.15g %20.15g"

# global thermo output to log
thermo_style custom step temp pe etotal pxx pxy pxz pyy pyz pzz vol
thermo_modify format float %20.15g
thermo 10

min_style   cg
minimize    0 0.01 1000000 1000000
"""
from ase.io.lammpsdata import read_lammps_data, write_lammps_data
from pymatgen.core import Structure
from pymatgen.io.lammps.data import LammpsData
from pymatgen.io.ase import AseAtomsAdaptor

working_directory = os.getcwd()
atoms = bulk("Fe") * [10,10,10]
atoms.rattle(0.3)
structure = AseAtomsAdaptor.get_structure(atoms)
lammps_data = LammpsData.from_structure(
    structure,
    atom_style="atomic",
)
os.makedirs(working_directory, exist_ok=True)
lammps_data.write_file("demo_pyiron_lammps/lammps_with_masses.data")
# 2) Assign chemical symbols so ASE knows which mass to use
#    Here all atoms are Fe:
atoms.set_chemical_symbols(['Fe'] * len(atoms))
# 3) Write out a new data file, sorted by type, with a Masses section
potential_elements = ["Fe"]
with open(os.path.join(working_directory, "lmp.in"), "w") as f:
    f.writelines(lammps_input_script)

command = "mpirun -np 40 --bind-to none /cmmc/ptmp/hmai/LAMMPS/lammps_grace/build/lmp -in lmp.in -log minimize.log"

output = subprocess.check_output(
    command,
    cwd=working_directory,
    shell=True,
    universal_newlines=True,
    env=os.environ.copy(),
)

output = parse_lammps_output_files(
    working_directory=working_directory,
    structure=atoms,
    potential_elements=potential_elements,
    units="metal",
    dump_h5_file_name="dump.h5",
    dump_out_file_name="dump.out",
    log_lammps_file_name="minimize.log",
)
output

2025-05-26 19:47:53.030543: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-26 19:47:53.030532: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-26 19:47:53.030551: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-26 19:47:53.035677: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machi

{'generic': {'eng': array([[-6.75575207, -6.72316516, -7.54244379, ..., -7.72358715,
          -7.61678979, -7.45744122],
         [-8.11920554, -8.14203334, -8.14581791, ..., -8.14692011,
          -8.14301978, -8.22608757],
         [-8.23136612, -8.22559122, -8.22854639, ..., -8.22281797,
          -8.23045368, -8.23411899],
         ...,
         [-8.22675499, -8.22704935, -8.22646648, ..., -8.22651629,
          -8.22698464, -8.22716539],
         [-8.22675528, -8.22705034, -8.22646711, ..., -8.22651417,
          -8.22698411, -8.22716571],
         [-8.22675747, -8.22705574, -8.22647044, ..., -8.22650307,
          -8.22698135, -8.22716716]]),
  'steps': array([  0,  10,  20,  30,  40,  50,  60,  70,  80,  90, 100, 110, 120,
         130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250,
         260]),
  'natoms': array([1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,
         1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,
         1000.,